<a href="https://colab.research.google.com/github/Disaster4255/PL_REPO_PY_Hao/blob/main/ProgrammingLanguage114_1_HW3%264.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 番茄鐘代辦事項與爬蟲
新增"刪除任務功能"與"以CSV或JSON匯出功能"<br>
爬蟲部分改為擷取後直接分析文本並顯示前三熱詞，再輸出AI簡短結論


In [3]:
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
               gradio pandas beautifulsoup4 google-generativeai python-dateutil

In [6]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup
from collections import Counter # 新增：用於詞頻統計

import google.generativeai as genai
from google.generativeai.types import GenerationConfig # 引入 GenerationConfig
import jieba # 新增：中文分詞庫

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default


from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
# 確保您已經在 Colab Secrets 中設定了 'gemini_key'
api_key = userdata.get('gemini_key')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

# 檢查 API key 是否存在，如果不存在，則會是 None 或空字串
API_KEY_IS_CONFIGURED = bool(api_key)

# 統一使用 flash 模型以加快速度和提升穩定性
GEMINI_MODEL = 'gemini-2.5-flash'


SHEET_URL = "https://docs.google.com/spreadsheets/d/162pRL7bo8TP6n_ebBqC8pDl_dlVbXab09XYwRoAHpoQ/edit?usp=sharing"
WORKSHEET_NAME = "工作表4"
TIMEZONE = "Asia/Taipei"


import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url(SHEET_URL)


# 從 gsheets 的 All-whiteboard-device 載入 sheets
sh = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sh[1:], columns=sh[0])
# 取得最前面的5筆資料
df.head()


# 中文基本停用詞列表 (用於 Jieba 分詞後過濾)
CHINESE_STOP_WORDS = set([
    '的', '得', '地', '是', '有', '在', '我', '你', '他', '她', '它', '們', '這', '那',
    '個', '了', '和', '與', '或', '但', '而', '則', '也', '都', '不', '很', '將', '會',
    '可以', '能夠', '因為', '所以', '雖然', '但是', '然而', '因此', '為了', '以便', '以致於',
    '對於', '關於', '由於', '至於', '直到', '從', '向', '給', '被', '讓', '叫', '將', '把',
    '對', '為', '時', '日', '年', '月', '號', '分', '秒', '如', '等', '之', '所', '其',
    '一', '二', '三', '四', '五', '六', '七', '八', '九', '十', '千', '萬', '億', '元',
    '人民幣', '美元', '英鎊', '歐元', '日圓', '是', '說', '這', '個', '人', '什麼', '怎麼',
    '一個', '一些', '一些', '一些', '一些', '一些', '沒有', '不可', '就是', '一個',
    # 標點符號及空白
    '，', '。', '、', '？', '！', '：', '；', '“', '”', '《', '》', '（', '）', '【', '】',
    '[', ']', '{', '}', '-', '_', '=', '+', '*', '/', '\\', '|', '<', '>', '~', '`',
    '.', ',', '!', '?', ';', ':', '"', "'", '...', '..', ' ', '\n', '\t', '\r',
    '’', '“', '”', '【', '】', '「', '」', '...', '⋯⋯', '—', '──', '...',
])


def ensure_spreadsheet(name):
    try:
        sh = gc.open(name)  # returns gspread.models.Spreadsheet
    except gspread.SpreadsheetNotFound:
        sh = gc.create(name)
    return sh

sh = ensure_spreadsheet(WORKSHEET_NAME)


def ensure_worksheet(sh, title, header):
    try:
        ws = sh.worksheet(title)
    except gspread.WorksheetNotFound:
        ws = sh.add_worksheet(title=title, rows="1000", cols=str(len(header)+5))
        ws.update([header])
    # 若沒有表頭就補上
    data = ws.get_all_values()
    if not data or (data and data[0] != header):
        ws.clear()
        ws.update([header])
    return ws

TASKS_HEADER = [
    "id","task","status","priority","est_min","start_time","end_time",
    "actual_min","pomodoros","due_date","labels","notes",
    "created_at","updated_at","completed_at","planned_for"
]
LOGS_HEADER = [
    "log_id","task_id","phase","start_ts","end_ts","minutes","cycles","note"
]
CLIPS_HEADER = ["clip_id","url","selector","text","href","created_at","added_to_task"]
# 新增分析結果工作表頭
ANALYSIS_HEADER = ["timestamp", "hot_word_1", "hot_word_2", "hot_word_3", "word_list_json", "gemini_conclusion"]


ws_tasks = ensure_worksheet(sh, "tasks", TASKS_HEADER)
ws_logs  = ensure_worksheet(sh, "pomodoro_logs", LOGS_HEADER)
ws_clips = ensure_worksheet(sh, "web_clips", CLIPS_HEADER)
ws_analysis = ensure_worksheet(sh, "keyword_analysis", ANALYSIS_HEADER) # 新工作表

def tznow():
    return dt.now(gettz(TIMEZONE))

def read_df(ws, header):
    df = get_as_dataframe(ws, evaluate_formulas=True, header=0)
    if df is None or df.empty:
        return pd.DataFrame(columns=header)
    df = df.fillna("")
    # 保證欄位齊全
    for c in header:
        if c not in df.columns:
            df[c] = ""
    # 型別微調
    if "est_min" in df.columns:
        df["est_min"] = pd.to_numeric(df["est_min"], errors="coerce").fillna(0).astype(int)
    if "actual_min" in df.columns:
        df["actual_min"] = pd.to_numeric(df["actual_min"], errors="coerce").fillna(0).astype(int)
    if "pomodoros" in df.columns:
        df["pomodoros"] = pd.to_numeric(df["pomodoros"], errors="coerce").fillna(0).astype(int)
    return df[header]

def write_df(ws, df, header):
    if df.empty:
        ws.clear()
        ws.update([header])
        return
    # 轉字串避免 gspread 型別問題
    df_out = df.copy()
    for c in df_out.columns:
        df_out[c] = df_out[c].astype(str)
    ws.clear()
    ws.update([header] + df_out.values.tolist())

def refresh_all():
    return (
        read_df(ws_tasks, TASKS_HEADER).copy(),
        read_df(ws_logs, LOGS_HEADER).copy(),
        read_df(ws_clips, CLIPS_HEADER).copy(),
        read_df(ws_analysis, ANALYSIS_HEADER).copy() # 新增載入分析結果
    )

tasks_df, logs_df, clips_df, analysis_df = refresh_all() # 更新載入變數

# ====================================================================
# 番茄鐘階段追蹤變數 (必須在全局範圍定義)
current_task_phase = {} # 用來追蹤任務的狀態：{"task_id": {"phase": "work", "start_ts": "..."}}
# ====================================================================

def add_task(task, priority, est_min, due_date, labels, notes, planned_for):
    global tasks_df
    _now = tznow().isoformat()
    new = pd.DataFrame([{
        "id": str(uuid.uuid4())[:8],
        "task": task.strip(),
        "status": "todo",
        "priority": priority or "M",
        "est_min": int(est_min) if est_min else 25,
        "start_time": "",
        "end_time": "",
        "actual_min": 0,
        "pomodoros": 0,
        "due_date": due_date or "",
        "labels": labels or "",
        "notes": notes or "",
        "created_at": _now,
        "updated_at": _now,
        "completed_at": "",
        "planned_for": planned_for or ""  # 可填 today / tomorrow / 空白
    }])
    tasks_df = pd.concat([tasks_df, new], ignore_index=True)
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return "✅ 已新增任務", tasks_df


def update_task_status(task_id, new_status):
    global tasks_df
    idx = tasks_df.index[tasks_df["id"] == task_id]
    if len(idx)==0:
        return "⚠️ 找不到任務", tasks_df
    i = idx[0]
    tasks_df.loc[i, "status"] = new_status
    tasks_df.loc[i, "updated_at"] = tznow().isoformat()
    if new_status == "done" and not tasks_df.loc[i, "completed_at"]:
        tasks_df.loc[i, "completed_at"] = tznow().isoformat()
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return "✅ 狀態已更新", tasks_df

def mark_done(task_id):
    return update_task_status(task_id, "done")


def delete_task(task_id):
    global tasks_df
    idx = tasks_df.index[tasks_df["id"] == task_id]
    if len(idx)==0:
        # 回傳 5 個值，空值用 gr.update() 來維持現狀
        return "⚠️ 找不到任務", tasks_df, gr.update(), gr.update(), gr.update()

    # 刪除任務
    tasks_df = tasks_df.drop(idx).reset_index(drop=True)
    write_df(ws_tasks, tasks_df, TASKS_HEADER)

    # 因為任務被刪除了，列表可能改變，回傳新的任務選項列表
    new_choices = list_task_choices()

    # 回傳 5 個輸出值：訊息、Dataframe、task_choice 更新、sel_task 更新、task_choice_del 更新
    return (
        "🗑️ 任務已刪除",
        tasks_df,
        gr.update(choices=new_choices),
        gr.update(choices=new_choices),
        gr.update(choices=new_choices)
    )


def recalc_task_actuals(task_id):
    """根據 logs_df 回寫 actual_min 與 pomodoros"""
    global tasks_df, logs_df
    # 確保 logs_df 是最新的，因為 end_phase 剛寫入
    logs_df = read_df(ws_logs, LOGS_HEADER)

    work_logs = logs_df[(logs_df["task_id"]==task_id) & (logs_df["phase"]=="work")]
    total_min = work_logs["minutes"].astype(float).sum() if not work_logs.empty else 0
    pomos = int(round(total_min / 25.0))
    idx = tasks_df.index[tasks_df["id"]==task_id]
    if len(idx)==0: return
    i = idx[0]
    tasks_df.loc[i,"actual_min"] = int(total_min)
    tasks_df.loc[i,"pomodoros"] = pomos
    tasks_df.loc[i,"updated_at"] = tznow().isoformat()

def list_task_choices():
    global tasks_df
    if tasks_df.empty:
        return []
    # 顯示： [status] (P:priority) task  — id
    def row_label(r):
        return f"[{r['status']}] (P:{r['priority']}) {r['task']} — {r['id']}"
    return [(row_label(r), r["id"]) for _, r in tasks_df.iterrows()]

# ====================================================================
# 【補齊】番茄鐘核心功能：Start Phase
# ====================================================================
def start_phase(task_id, phase, cycles):
    global current_task_phase, tasks_df
    if not task_id:
        return gr.update(value="⚠️ 請選擇一個任務開始。")
    if task_id in current_task_phase:
        # 顯示當前進行中的任務名稱
        task_name = current_task_phase[task_id].get("task_name", f"ID: {task_id}")
        return gr.update(value=f"⚠️ 任務 {task_name} 仍在進行中：{current_task_phase[task_id]['phase']}。")

    _now = tznow().isoformat()

    # 獲取任務名稱
    task_row = tasks_df[tasks_df["id"]==task_id]
    task_name = task_row["task"].iloc[0] if not task_row.empty else f"ID: {task_id}"

    current_task_phase[task_id] = {
        "phase": phase,
        "start_ts": _now,
        "cycles": int(cycles) if phase == "work" else 0, # 只有工作才計數
        "task_name": task_name
    }

    if phase == "work":
        idx = tasks_df.index[tasks_df["id"] == task_id]
        if len(idx)>0:
            # 更新任務的 start_time 和 status
            if not tasks_df.loc[idx[0], "start_time"]:
                tasks_df.loc[idx[0], "start_time"] = _now
            if tasks_df.loc[idx[0], "status"] != "done":
                tasks_df.loc[idx[0], "status"] = "in-progress"
                tasks_df.loc[idx[0], "updated_at"] = _now
                write_df(ws_tasks, tasks_df, TASKS_HEADER)
            return gr.update(value=f"🚀 任務：{task_name} | 開始 **工作**，番茄數：{cycles}。")
    else:
        return gr.update(value=f"🍵 任務：{task_name} | 開始 **休息**。")

# ====================================================================
# 【補齊】番茄鐘核心功能：End Phase
# ====================================================================
def end_phase(task_id, note):
    global current_task_phase, logs_df, tasks_df

    if task_id not in current_task_phase:
        return gr.update(value="⚠️ 找不到進行中的階段，請先點擊「開始工作/休息」。")

    phase_data = current_task_phase.pop(task_id) # 移除階段
    _now = tznow()
    # 使用 fromisoformat 轉換時間
    try:
        start_ts = dt.fromisoformat(phase_data["start_ts"])
    except ValueError:
        # 如果時間格式有誤，給一個預設值，避免崩潰
        start_ts = _now - timedelta(minutes=1)

    # 計算時長
    duration = (_now - start_ts)
    minutes = round(duration.total_seconds() / 60)

    # 讀取 logs_df 以便附加新的 Log
    logs_df = read_df(ws_logs, LOGS_HEADER)

    # 寫入 Log
    new_log = pd.DataFrame([{
        "log_id": str(uuid.uuid4())[:8],
        "task_id": task_id,
        "phase": phase_data["phase"],
        "start_ts": phase_data["start_ts"],
        "end_ts": _now.isoformat(),
        "minutes": minutes,
        "cycles": phase_data["cycles"],
        "note": note or ""
    }])
    logs_df = pd.concat([logs_df, new_log], ignore_index=True)
    write_df(ws_logs, logs_df, LOGS_HEADER)

    # 如果是 work，更新任務的實際時間和番茄數
    if phase_data["phase"] == "work":
        # 更新 tasks_df 中的實際時間和番茄數，並寫回 Sheet
        recalc_task_actuals(task_id)
        write_df(ws_tasks, tasks_df, TASKS_HEADER)

        # 重新載入 logs_df (雖然已經在上面寫入，但確保 Dataframe 是最新的)
        logs_df_latest = read_df(ws_logs, LOGS_HEADER).copy()

        # 輸出 2 個值：訊息 (msg_pomo), Dataframe (grid_logs)
        return gr.update(value=f"✅ 任務：{phase_data['task_name']} | 結束 **工作**，記錄 {minutes} 分鐘。"), logs_df_latest
    else:
        logs_df_latest = read_df(ws_logs, LOGS_HEADER).copy()
        return gr.update(value=f"🧘 任務：{phase_data['task_name']} | 結束 **休息**，記錄 {minutes} 分鐘。"), logs_df_latest
# ====================================================================

# AI 計畫（Gemini；無金鑰則規則式）
def generate_today_plan():
    global tasks_df
    # 以「due_date 是今天」或「planned_for = today」且不是 done 的任務為計畫清單
    today = tznow().date().isoformat()
    cand = tasks_df[
        ((tasks_df["due_date"]==today) | (tasks_df["planned_for"].str.lower()=="today")) &
        (tasks_df["status"]!="done")
    ].copy()
    if cand.empty:
        return "📭 今天沒有標記的任務。請在 Tasks 分頁把任務的 due_date 設為今天或 planned_for 設為 today。"

    # 先依 priority（H>M>L）+ est_min 排序
    pr_order = {"H":0, "M":1, "L":2}
    cand["p_ord"] = cand["priority"].map(pr_order).fillna(3)
    cand = cand.sort_values(["p_ord","est_min"], ascending=[True, True])

    # 檢查 API_KEY_IS_CONFIGURED (使用全局變量)
    plan_md = "" # 先初始化為空

    if API_KEY_IS_CONFIGURED:
        sys_prompt = (
            "你是一位任務規劃助理。請把輸入的任務（含估時與優先級）排成三段：morning、afternoon、evening，"
            "並給出每段的重點、順序、每項的時間預估與備註。總時數請大致符合任務估時總和。"
            "回傳以 Markdown 條列，格式：\n"
            "### Morning\n- [任務ID] 任務名稱（預估 xx 分）— 備註\n..."
            "### Afternoon\n...\n### Evening\n...\n"
        )
        items = []
        for _, r in cand.iterrows():
            items.append({
                "id": r["id"], "task": r["task"], "est_min": int(r["est_min"]),
                "priority": r["priority"]
            })
        user_content = json.dumps({"today": today, "tasks": items}, ensure_ascii=False)

        # 使用 GenerationConfig 傳遞 System Instruction
        config = GenerationConfig(
            system_instruction=sys_prompt
        )

        try:
            model = genai.GenerativeModel(GEMINI_MODEL)
            # 將系統提示從 generate_content 移除，改用 config 參數
            resp = model.generate_content(user_content, config=config)
            plan_md = resp.text
        except Exception as e:
            # 確保錯誤信息對開發者有幫助
            plan_md = f"⚠️ Gemini 失敗（Plan）：{e}\n\n改用規則式規劃。"
            print(f"Gemini Plan Error: {e}")
    else:
        plan_md = "🔧 未設定 GEMINI_API_KEY，使用規則式規劃。\n\n"

    # 規則式：把高優先任務平均切到上午/下午/晚上
    buckets = {"morning": [], "afternoon": [], "evening": []}

    # 使用優先級 H->M->L 順序放入 bucket
    priority_tasks = cand.sort_values("p_ord", ascending=True)
    bucket_keys = ["morning", "afternoon", "evening"]

    for i, (_, r) in enumerate(priority_tasks.iterrows()):
        bucket_name = bucket_keys[i % 3]
        buckets[bucket_name].append(r)

    def sec_md(name, rows):
        if not rows: return f"### {name.title()}\n（無）\n"
        lines = [f"### {name.title()}"]
        for r in rows:
            lines.append(f"- [{r['id']}] {r['task']}（預估 {int(r['est_min'])} 分，P:{r['priority']}）")
        return "\n".join(lines) + "\n"

    rule_md = sec_md("morning", buckets["morning"]) + "\n" + \
              sec_md("afternoon", buckets["afternoon"]) + "\n" + \
              sec_md("evening", buckets["evening"])

    # 只有在 AI 輸出不成功，或 API 未配置時，才顯示規則式結果
    if not API_KEY_IS_CONFIGURED or plan_md.startswith("⚠️ Gemini 失敗"):
        return (plan_md + "\n---\n**【規則式規劃結果】**\n" + rule_md).strip()
    else:
        # 如果 AI 成功，則 AI 輸出在前，規則式結果在分隔線後
        return (plan_md + "\n\n---\n**【規則式規劃結果】**\n" + rule_md).strip()


# 今日完成率
def today_summary():
    global tasks_df
    today = tznow().date().isoformat()
    planned = tasks_df[
        ((tasks_df["due_date"]==today) | (tasks_df["planned_for"].str.lower()=="today"))
    ]
    total = len(planned)

    if total == 0:
        return "📅 今日計畫任務：0；✅ 完成：0；📈 完成率：0.0%"

    done = planned[planned["status"]=="done"]
    done_n = len(done)
    rate = (done_n/total*100)
    return f"📅 今日計畫任務：{total}；✅ 完成：{done_n}；📈 完成率：{rate:.1f}%"


# 匯出
def export_data(table_name, format_type):
    global tasks_df, logs_df, clips_df, analysis_df

    # 選擇對應的 DataFrame
    if table_name == "Tasks":
        df = tasks_df
    elif table_name == "Pomodoro Logs":
        df = logs_df
    elif table_name == "Web Clips":
        df = clips_df
    elif table_name == "Keyword Analysis":
        df = analysis_df # 新增匯出分析結果
    else:
        # 如果沒有選中任何項目，回傳 None 和錯誤訊息
        return None, "⚠️ 請選擇要匯出的資料表"

    # 處理空的 DataFrame
    if df.empty:
        # Gradio 的 gr.File 需要回傳 None 來清除/不提供檔案
        return None, f"⚠️ {table_name} 資料表為空，無法匯出"

    # 決定檔案名稱和內容
    # 使用日期時間確保檔案名稱唯一性
    base_filename = f"{table_name.lower().replace(' ', '_')}_{dt.now().strftime('%Y%m%d_%H%M%S')}"

    if format_type == "CSV":
        # 儲存為 CSV 檔案
        filepath = f"{base_filename}.csv"
        # 使用 encoding='utf-8-sig' 確保匯出的 CSV 在 Excel 中開啟時中文不亂碼
        df.to_csv(filepath, index=False, encoding='utf-8-sig')
        return filepath, f"✅ 已匯出 {len(df)} 筆資料至 {filepath} (CSV 格式)"

    elif format_type == "JSON":
        # 儲存為 JSON 檔案
        filepath = f"{base_filename}.json"
        # orient='records' 會將 DataFrame 轉為適合的 JSON 列表格式
        df.to_json(filepath, orient='records', force_ascii=False)
        return filepath, f"✅ 已匯出 {len(df)} 筆資料至 {filepath} (JSON 格式)"

    else:
        return None, "⚠️ 請選擇有效的匯出格式"

# =========================
# 爬蟲：擷取文字或連結並可加入任務 (已移除加入任務功能)
# =========================
def crawl(url, selector, mode, limit):
    try:
        # 增加 timeout 確保不會無限期等待
        resp = requests.get(url, timeout=15, headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"})
        resp.raise_for_status() # 檢查 HTTP 狀態碼 (200 OK 以外的都拋出異常)
    except requests.exceptions.RequestException as e:
        # 捕捉所有 requests 相關的異常 (如連線失敗, timeout, 4xx/5xx HTTP 錯誤)
        error_msg = f"⚠️ 請求失敗 ({url})：{type(e).__name__} - {e}"
        print(error_msg)
        return pd.DataFrame(columns=CLIPS_HEADER), error_msg
    except Exception as e:
        error_msg = f"⚠️ 請求失敗：意外錯誤 - {e}"
        print(error_msg)
        return pd.DataFrame(columns=CLIPS_HEADER), error_msg

    # 處理成功的回應
    try:
        soup = BeautifulSoup(resp.text, "html.parser")
        nodes = soup.select(selector)
    except Exception as e:
        return pd.DataFrame(columns=CLIPS_HEADER), f"⚠️ 解析錯誤：無效的 Selector 或 HTML 結構 - {e}"

    rows = []
    # 確保 limit 是有效的整數，如果不是，則使用預設值 20
    try:
        limit_n = int(limit) if limit else 20
    except ValueError:
        limit_n = 20

    for i, n in enumerate(nodes[:limit_n]):
        text = n.get_text(strip=True) if mode in ("text","both") else ""
        href = n.get("href") if mode in ("href","both") else ""
        # 相對連結處理
        if href and href.startswith("/"):
            from urllib.parse import urljoin
            # 確保 urljoin 的 base url 不包含 / 的路徑部分
            import urllib.parse
            parsed_url = urllib.parse.urlparse(url)
            base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
            href = urljoin(base_url, href)

        rows.append({
            "clip_id": str(uuid.uuid4())[:8],
            "url": url,
            "selector": selector,
            "text": text,
            "href": href,
            "created_at": tznow().isoformat(),
            "added_to_task": "no" # 保留欄位但固定填 no
        })
    df = pd.DataFrame(rows, columns=CLIPS_HEADER)

    if df.empty:
        return df, f"⚠️ 擷取 0 筆。請檢查 Selector '{selector}' 是否正確。"

    return df, f"✅ 成功擷取 {len(df)} 筆"


# 新增功能：分析文本，產生熱詞與 AI 結論
def analyze_text_and_conclude(texts):
    global ws_analysis

    if not texts:
        return "（無文本）", "⚠️ 無文本可供分析"

    # 1. 文本合併與分詞
    full_text = " ".join(texts)
    # 使用 Jieba 進行精確模式分詞
    seg_list = jieba.cut(full_text, cut_all=False)

    # 2. 過濾停用詞和單字詞
    words = [
        word.strip().lower() for word in seg_list
        if word.strip() and len(word.strip()) > 1 and word.strip() not in CHINESE_STOP_WORDS and not word.isdigit()
    ]

    # 3. 詞頻統計 (作為 TF-IDF 的簡易替代)
    word_counts = Counter(words)
    top_3 = word_counts.most_common(3)

    # 檢查詞數是否足夠
    if len(top_3) < 3:
        hot_words = [f"{item[0]} ({item[1]})" for item in top_3] + ["(N/A)"] * (3 - len(top_3))
        conclusion = "⚠️ 文本詞彙量不足，無法生成有意義的學術結論。"
        top_3_clean = [item[0] for item in top_3] + [""] * (3 - len(top_3))
    else:
        hot_words = [f"{item[0]} ({item[1]})" for item in top_3]
        hot_words_str = "、".join([item[0] for item in top_3])
        top_3_clean = [item[0] for item in top_3]

        # 4. 生成 AI 結論
        conclusion = "⚠️ Gemini 結論生成失敗"

        # 檢查 API_KEY_IS_CONFIGURED
        if not API_KEY_IS_CONFIGURED:
            conclusion = "🔧 未設定 GEMINI_API_KEY，無法生成結論。"
            return f"🔥 熱詞：{hot_words[0]}、{hot_words[1]}、{hot_words[2]}", conclusion

        # 使用 GenerationConfig 傳遞 System Instruction
        sys_prompt = (
            "你是一位資深的新聞媒體研究學者。請針對以下熱詞，結合當前媒體趨勢和公共討論的脈絡，"
            f"撰寫一篇約120字的精簡結論。請使用繁體中文，並保持專業、客觀的學術分析語氣。"
            f"避免過度引用熱詞本身，而是分析它們背後的社會意義和媒體現象。"
        )
        user_content = f"請分析這三個熱詞：{hot_words_str}"

        config = GenerationConfig(
            system_instruction=sys_prompt
        )

        try:
            model = genai.GenerativeModel(GEMINI_MODEL)
            resp = model.generate_content(user_content, config=config)
            conclusion = resp.text.strip()
        except Exception as e:
            conclusion = f"⚠️ Gemini 結論生成失敗：{e}"
            print(f"Gemini Analysis Error: {e}")

    # 5. 寫入分析結果到新工作表 (keyword_analysis)
    _now = tznow().isoformat()
    new_row = pd.DataFrame([{
        "timestamp": _now,
        "hot_word_1": top_3_clean[0],
        "hot_word_2": top_3_clean[1],
        "hot_word_3": top_3_clean[2],
        # 儲存前 20 個詞頻最高的詞彙及其計數
        "word_list_json": json.dumps(word_counts.most_common(20), ensure_ascii=False),
        "gemini_conclusion": conclusion
    }])

    # 讀取現有分析數據並附加新結果
    global analysis_df
    # 確保 analysis_df 是最新的，因為我們需要在它後面追加
    analysis_df = read_df(ws_analysis, ANALYSIS_HEADER)
    analysis_df = pd.concat([analysis_df, new_row], ignore_index=True)
    write_df(ws_analysis, analysis_df, ANALYSIS_HEADER)
    # 返回熱詞字串和結論
    return f"🔥 熱詞：{hot_words[0]}、{hot_words[1]}、{hot_words[2]}", conclusion

def _crawl_analyze_and_save(u, s, m, l):
    """執行爬蟲，保存結果，並進行文本分析和結論生成"""
    df, msg = crawl(u, s, m, l)

    # 寫入 web_clips
    global clips_df
    clips_df = df.copy() # <--- 關鍵：使用 copy() 確保獨立的 Dataframe 供 Gradio 顯示
    write_df(ws_clips, clips_df, CLIPS_HEADER)

    hot_words_str = "（無）"
    conclusion = "⚠️ 擷取結果中無有效文本，無法進行分析。"

    # 進行文本分析與結論生成
    if not df.empty:
        # 篩選出有文本的項目
        texts_to_analyze = [r['text'] for _, r in df.iterrows() if r['text'].strip()]

        if texts_to_analyze:
            # analyze_text_and_conclude 會更新全局的 analysis_df
            hot_words_str, conclusion = analyze_text_and_conclude(texts_to_analyze)
            msg += f"。已成功分析 {len(texts_to_analyze)} 筆文本。"
        else:
             msg += "。擷取結果中無有效文本可供分析。"
    else:
        msg += "。無文本可供分析。"

    # 更新 Analysis Dataframe 顯示
    global analysis_df
    # 這裡必須重新從 Sheet 讀取，因為 analyze_text_and_conclude 已經寫入並更新了 analysis_df
    # 為了 Gradio 顯示，我們確保返回的都是最新的 Dataframe 副本
    analysis_df = read_df(ws_analysis, ANALYSIS_HEADER)

    # 返回 5 個值，確保 Dataframe 使用 .copy()
    return msg, clips_df.copy(), hot_words_str, conclusion, analysis_df.copy()

# =========================
# Gradio 介面
# =========================
def _refresh():
    global tasks_df, logs_df, clips_df, analysis_df
    tasks_df, logs_df, clips_df, analysis_df = refresh_all()
    choices = list_task_choices()
    # 回傳 8 個輸出：grid_tasks, grid_logs, grid_clips, grid_analysis, task_choice, out_summary, sel_task, task_choice_del
    return (
        tasks_df.copy(),
        logs_df.copy(),
        clips_df.copy(),
        analysis_df.copy(),
        gr.update(choices=choices),
        today_summary(),
        gr.update(choices=choices),
        gr.update(choices=choices)
    )

with gr.Blocks(title="待辦清單＋番茄鐘＋AI 計畫（Sheet/Gradio/爬蟲）") as demo:
    gr.Markdown("# ✅ 待辦清單與番茄鐘（Google Sheet＋Gradio＋Crawler＋AI 計畫）")
    with gr.Row():
        btn_refresh = gr.Button("🔄 重新整理（Sheet → App）")
        out_summary = gr.Markdown(today_summary())

    with gr.Tab("Tasks"):
        with gr.Row():
            with gr.Column(scale=2):
                task = gr.Textbox(label="任務名稱", placeholder="寫 HW3 報告 / 修正 SQL / …")
                priority = gr.Dropdown(["H","M","L"], value="M", label="優先級")
                est_min = gr.Number(value=25, label="預估時間（分鐘）", precision=0)
                due_date = gr.Textbox(label="到期日（YYYY-MM-DD，可空白）")
                labels = gr.Textbox(label="標籤（逗號分隔，可空白）")
                notes = gr.Textbox(label="備註（可空白）")
                planned_for = gr.Dropdown(["","today","tomorrow"], value="", label="規劃歸屬")
                btn_add = gr.Button("➕ 新增任務")
                msg_add = gr.Markdown()
            with gr.Column(scale=3):
                # 確保 Dataframe 使用 copy()
                grid_tasks = gr.Dataframe(value=tasks_df.copy(), label="任務清單（直接從 Sheet 來）", interactive=False)

        with gr.Row():
            task_choice = gr.Dropdown(choices=list_task_choices(), label="選取任務（用於更新）")
            new_status = gr.Dropdown(["todo","in-progress","done"], value="in-progress", label="更新狀態")
            btn_update = gr.Button("✏️ 更新狀態")
            btn_done = gr.Button("✅ 直接標記完成")

        with gr.Row():
            task_choice_del = gr.Dropdown(choices=list_task_choices(), label="選取任務（用於刪除）")
            btn_delete = gr.Button("🗑️ 刪除該任務")
            msg_delete = gr.Markdown()


        msg_update = gr.Markdown()

    with gr.Tab("Pomodoro"):
        gr.Markdown("## 🍅 番茄鐘計時與紀錄")
        with gr.Row():
            sel_task = gr.Dropdown(choices=list_task_choices(), label="選擇任務")
            cycles = gr.Number(value=1, precision=0, label="番茄數（僅作紀錄）")
        with gr.Row():
            btn_start_work = gr.Button("▶️ 開始工作 (25min)")
            note_work = gr.Textbox(label="工作備註（可空白）")
            btn_end_work = gr.Button("⏹️ 結束工作並記錄")
        with gr.Row():
            btn_start_break = gr.Button("🍵 開始休息 (5min)")
            note_break = gr.Textbox(label="休息備註（可空白）")
            btn_end_break = gr.Button("⏹️ 結束休息並記錄")
        # 這裡的 msg_pomo 接收 start_phase 和 end_phase 的回傳值
        msg_pomo = gr.Markdown()
        # 確保 Dataframe 使用 copy()
        grid_logs = gr.Dataframe(value=logs_df.copy(), label="番茄鐘紀錄", interactive=False)

    with gr.Tab("AI Plan"):
        # 顯示 API 狀態，提供更清晰的指引
        api_status = "✅ API 金鑰已配置" if API_KEY_IS_CONFIGURED else "❌ 警告：API 金鑰未配置，將使用規則式規劃。"
        gr.Markdown(f"## 🧠 今日 AI 行動計畫\n**模型：{GEMINI_MODEL}**\n{api_status}\n把**今天的任務**排成 **morning / afternoon / evening** 三段行動計畫。")
        btn_plan = gr.Button("🧠 產生今日計畫")
        out_plan = gr.Markdown()

    with gr.Tab("Crawler"):
        gr.Markdown("## 🕷️ 網站擷取與熱詞分析")
        url = gr.Textbox(label="目標 URL", placeholder="https://example.com/news", value="https://tw.news.yahoo.com/")
        selector = gr.Textbox(label="CSS Selector", placeholder="h3.Mb\\(5px\\) a / a", value="h3.Mb\\(5px\\) a")
        mode = gr.Radio(["text","href","both"], value="text", label="擷取內容")
        limit = gr.Number(value=20, precision=0, label="最多擷取幾筆")
        btn_crawl = gr.Button("🕷️ 開始擷取與分析")
        msg_crawl = gr.Markdown()
        # 確保 Dataframe 使用 copy()
        grid_clips = gr.Dataframe(value=clips_df.copy(), label="擷取結果（只保留最新結果，會同步寫入 Web Clips 工作表）", interactive=False)

        gr.Markdown("---")
        gr.Markdown("### 📰 文本分析與學術結論")
        out_hot_words = gr.Textbox(label="前 3 大熱詞 (詞彙 (詞頻))", interactive=False)
        out_gemini_conclusion = gr.Markdown(label="新聞媒體研究學者結論（已寫入 keyword_analysis 工作表）")


    with gr.Tab("Analysis Results"): # 新增一個 Tab 來顯示分析結果的工作表內容
        gr.Markdown("## 📊 關鍵字分析歷史紀錄")
        # 確保 Dataframe 使用 copy()
        grid_analysis = gr.Dataframe(value=analysis_df.copy(), label="熱詞分析紀錄（來自 keyword_analysis 工作表）", interactive=False)


    with gr.Tab("Summary"):
        btn_summary = gr.Button("📊 重新計算今日完成率")
        out_summary2 = gr.Markdown()



    #匯出
    with gr.Tab("Export Data"):
        gr.Markdown("## 💾 資料匯出 (Export Data)")
        with gr.Row():
            export_table = gr.Radio(
                choices=["Tasks", "Pomodoro Logs", "Web Clips", "Keyword Analysis"], # 新增選項
                value="Tasks",
                label="選擇要匯出的資料表"
            )
            export_format = gr.Radio(
                choices=["CSV", "JSON"],
                value="CSV",
                label="選擇匯出格式"
            )
        btn_export = gr.Button("⬇️ 匯出資料")
        msg_export = gr.Markdown("選擇資料表和格式，然後點擊「匯出資料」。")
        # 使用 gr.File 作為輸出，讓使用者可以直接下載檔案
        file_output = gr.File(label="下載匯出的檔案", type="filepath")

    # === 綁定動作 ===
    # 更新 _refresh 綁定輸出
    btn_refresh.click(_refresh, outputs=[grid_tasks, grid_logs, grid_clips, grid_analysis, task_choice, out_summary, sel_task, task_choice_del])

    btn_add.click(
        add_task,
        inputs=[task, priority, est_min, due_date, labels, notes, planned_for],
        outputs=[msg_add, grid_tasks]
    )

    btn_update.click(
        update_task_status,
        inputs=[task_choice, new_status],
        outputs=[msg_update, grid_tasks]
    )

    btn_done.click(
        mark_done,
        inputs=[task_choice],
        outputs=[msg_update, grid_tasks]
    )

    btn_delete.click(
        delete_task,
        inputs=[task_choice_del],
        # 輸出：訊息 (msg_delete)，任務 Dataframe (grid_tasks)，並更新三個下拉選單 (task_choice, sel_task, task_choice_del)
        outputs=[msg_delete, grid_tasks, task_choice, sel_task, task_choice_del]
    )

    # 綁定番茄鐘功能 (修正輸出數量以包含 grid_logs)
    # Start Phase 只需要輸出訊息
    btn_start_work.click(
        start_phase, inputs=[sel_task, gr.State("work"), cycles], outputs=[msg_pomo]
    )
    # End Phase 需要輸出訊息和 logs Dataframe
    btn_end_work.click(
        end_phase, inputs=[sel_task, note_work], outputs=[msg_pomo, grid_logs]
    )
    # Start Phase 只需要輸出訊息
    btn_start_break.click(
        start_phase, inputs=[sel_task, gr.State("break"), cycles], outputs=[msg_pomo]
    )
    # End Phase 需要輸出訊息和 logs Dataframe
    btn_end_break.click(
        end_phase, inputs=[sel_task, note_break], outputs=[msg_pomo, grid_logs]
    )

    btn_plan.click(generate_today_plan, outputs=[out_plan])

    # 綁定匯出資料按鈕
    btn_export.click(
        export_data,
        inputs=[export_table, export_format],
        outputs=[file_output, msg_export] # 輸出檔案路徑到 gr.File，輸出訊息到 gr.Markdown
    )

    # 綁定新的爬蟲與分析函數 (確保輸出數量和順序正確)
    btn_crawl.click(
        _crawl_analyze_and_save,
        inputs=[url, selector, mode, limit],
        # 輸出順序：訊息, clips_df, hot_words_str, conclusion, analysis_df
        outputs=[msg_crawl, grid_clips, out_hot_words, out_gemini_conclusion, grid_analysis]
    )

    btn_summary.click(today_summary, outputs=[out_summary2])

demo.launch(share=True) # 建議開啟 share=True 讓您可以在外部測試

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3dda0ab4b7230d5c65.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


AttributeError: module 'gradio' has no attribute 'blocks'